# **Exploring different photorespiratory bypass in C3 diel leaf model**

In this part, we will discuss how FBA could be helpful for metabolic engineering. A recent study by [South et al.,2019](https://www.science.org/doi/10.1126/science.aat9077) explained three different photorespiratory bypasses in field-grown tobacco. In this exercise, we applied the three pathways to the PlantCoreMetabolism model and analysed the results for photosynthetic efficiency.

#### NOTE:

*   AP1 from the study represented as Pathway1 in this colab
*   AP2 from the study represented as Pathway2 in this colab
*   AP3 from the study represented as Pathway3 in this colab



#  **Part:1 Constraining C3 diel leaf model**

#Step:1 Install cobrapy and retrieve C3 model


*   Run `%pip install cobra` to install cobrapy
*   Use wget using the syntax `!wget link-to-file` to retrieve the models and functions required for this analysis
*  The given ` https:` link will get the  C3 diel leaf model from the github repository

In [2]:
%pip install cobra --quiet
!wget -q https://github.com/sshameer/DielLeafModeling/raw/refs/heads/main/Models/C3_model.sbml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 46.8 MB/s eta 0:00:00


#Step: 2 Read sbml model and perform pFBA

*   A cobra model can be generated from an SBML file using the` read_sbml_model`
function in the cobrapy io module
*   The syntax is `model_name = read_sbml_model(file_name)`
*   pFBA can be performed using the pfba package in the cobrapy `flux_analysis` module

In [3]:
#import model
from cobra.io import read_sbml_model
C3_model = read_sbml_model("C3_model.sbml")
from cobra.flux_analysis import pfba

#Step:3 Constraining the diel C3 leaf nodel
*   GPT transport reaction and starch phosphorylation are constraining.
*   Use the syntax `model_name.reactions.get_by_id` to get the reaction from the model
*   PPFD and and photon uptake defined as 500
*   Maintenanace cost is defined according the reference of [Töpfer et al.,2020](https://doi.org/10.1105/tpc.20.00132)
*   Sucrose transport reaction from day to night is blocked by setting the bounds to zero




In [4]:
# constraining GPT transport reaction
C3_model.reactions.get_by_id("G6P_Pi_pc1").lower_bound= 0
C3_model.reactions.get_by_id("G6P_Pi_pc1").upper_bound= 0
C3_model.reactions.get_by_id("G6P_Pi_pc2").lower_bound= 0
C3_model.reactions.get_by_id("G6P_Pi_pc2").upper_bound= 0
# constraining starch phosphorylation
C3_model.reactions.get_by_id("RXN_1826_p1").lower_bound= 0
C3_model.reactions.get_by_id("RXN_1826_p1").upper_bound= 0
C3_model.reactions.get_by_id("RXN_1826_p2").lower_bound= 0
C3_model.reactions.get_by_id("RXN_1826_p2").upper_bound= 0

# Setting photon uptake
PPFD = 500
C3_model.reactions.Photon_tx1.upper_bound = PPFD
C3_model.reactions.Photon_tx1.lower_bound = 0
# Setting maintenance cost
VATPase = 0.0049*PPFD+2.7851  # Töpfer et al.,2020
C3_model.reactions.get_by_id("ATPase_tx1").bounds = (VATPase,VATPase)
# constraining sucrose accumulation to model a starch storing leaf
C3_model.reactions.get_by_id("SUCROSE_v_dielTransfer").bounds = (0,0)


#Step:4 Optimization of model using pfba

*   pfba can be performed by using the syntax `solution_name = pfba(model_name)`
*   The solution object has an attribute fluxes which containts a dictionary of all fluxes
*   Use the syntax for checking the summary `model_name.summary()` of the model after optimization

In [5]:
#run pfba and get summary
solution = pfba(C3_model)
C3_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,35.14,1,100.00%
WATER_e1,H2O_tx1,31.55,0,0.00%
NITRATE_e1,Nitrate_tx1,0.2171,0,0.00%
NITRATE_e2,Nitrate_tx2,0.1447,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,2.023,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
SULFATE_e1,SO4_tx1,0.002766,0,0.00%
PROTON_c1,unlProtHYPO_c1,0.3151,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e2,CO2_tx2,-1.846,1,100.00%


# **Part:2 Checking alternative photorespiratory pathways one by one**

#Step:1 Appying Pathway 1 to the C3 diel leaf model
Pathway 1 represents AP1 in [South et al.,2019](https://www.science.org/doi/10.1126/science.aat9077) study. The enzymes glycolate dehydrogenase, glyoxylate carboligase and tartronic semialdehyde reductase make this pathway.
*   Making a copy of C3 model to apply AP1
*   To copy a cobra object using the `object.copy()` function
*   made a copy of C3 model for AP1 using the syntax `model_1 = C3_model.copy()`


In [6]:
# Making a copy of C3_model for AP1
model_1 = C3_model.copy()

# Disabling the  existing photorespiration pathway before allowing the alternate pathway by applying the following constraints
model_1.reactions.get_by_id("GLYCOLLATE_pc1").bounds = (0,0)
model_1.reactions.get_by_id("GLYCERATE_GLYCOLLATE_pc1").bounds = (0,0)
model_1.reactions.get_by_id("Glycolate_xc1").bounds = (0,0)

### Adding reactions to the model for AP1
*   creating an open list by using `reactions_to_add = []`.
*   Add the reactions in AP1 to both day and night
*   adding the reaction list to the model by `reactions_to_add.extend([R1,R2,R3...])` and `model_name.add_reactions(reactions_to_add)`



In [7]:
from cobra import Model, Reaction, Metabolite
# Create a list to hold reactions
reactions_to_add = []

# adding reactions for glycolate dehydrogenase for day
reaction1 = Reaction('RXN_GDH_p1')
reaction1.name = 'RXN_GDH_p1'
reaction1.subsystem = 'glycolate to glyoxyllate by NAD'
reaction1.lower_bound = 0
reaction1.upper_bound = 1000
GLYCOLLATE_p1 = model_1.metabolites.get_by_id("GLYCOLLATE_p1")
NAD_p1 = model_1.metabolites.get_by_id("NAD_p1")
GLYOX_p1 =Metabolite("GLYOX_p1",formula="C2H1O3", name= "Glyoxylate", compartment = "p", charge = "-1")
NADH_p1 = model_1.metabolites.get_by_id("NADH_p1")
reaction1.add_metabolites({
    GLYCOLLATE_p1: -1,
    NAD_p1: -1,
    GLYOX_p1: 1.0,
    NADH_p1: 1.0
})

# adding reactions for tartronate-semialdehyde synthase for day
reaction2 = Reaction('GCL_RXN_p1')
reaction2.name = 'GCL_RXN_p1'
reaction2.subsystem = 'glyoxyllate to TARTRONIC SEMIALDEHYDE'
reaction2.lower_bound = 0
reaction2.upper_bound = 1000
GLYOX_p1 =Metabolite("GLYOX_p1",formula="C2H1O3", name= "Glyoxylate", compartment = "p", charge = "-1")
PROTON_p1 = model_1.metabolites.get_by_id("PROTON_p1")
TARTRONIC_SEMIALDEHYDE_p1 = Metabolite("TARTRONIC_SEMIALDEHYDE_p1",formula="C3H3O4", name= "TARTRONIC_SEMIALDEHYDE", compartment = "p", charge = "-1")
CARBON_DIOXIDE_p1 = model_1.metabolites.get_by_id("CARBON_DIOXIDE_p1")
reaction2.add_metabolites({
    GLYOX_p1: -1,
    PROTON_p1: -1,
    TARTRONIC_SEMIALDEHYDE_p1: 1.0,
    CARBON_DIOXIDE_p1: 1.0,
})

# adding reactions for tartronate-semialdehyde reductase for day
reaction2 = Reaction('GCL_RXN_p1')
reaction3 = Reaction('RXN_TSR_p1')
reaction3.name = 'RXN_TSR_p1'
reaction3.subsystem = 'TARTRONIC SEMIALDEHYDE to glycerate'
reaction3.lower_bound = -1000
reaction3.upper_bound = 1000
TARTRONIC_SEMIALDEHYDE_p1 = Metabolite("TARTRONIC_SEMIALDEHYDE_p1",formula="C3H3O4", name= "TARTRONIC_SEMIALDEHYDE", compartment = "p", charge = "-1")
NADH_p1 = model_1.metabolites.get_by_id("NADH_p1")
NAD_p1 = model_1.metabolites.get_by_id("NAD_p1")
GLYCERATE_p1 = model_1.metabolites.get_by_id("GLYCERATE_p1")
reaction3.add_metabolites({
    TARTRONIC_SEMIALDEHYDE_p1: -1,
    NADH_p1: -1,
    GLYCERATE_p1: 1.0,
    NAD_p1: 1.0,
})

# adding reactions for glycolate dehydrogenase for night
reaction4 = Reaction('RXN_GDH_p2')
reaction4.name = 'RXN_GDH_p2'
reaction4.subsystem = 'glycolate to glyoxyllate by NAD'
reaction4.lower_bound = 0
reaction4.upper_bound = 1000
GLYCOLLATE_p2 = model_1.metabolites.get_by_id("GLYCOLLATE_p2")
NAD_p2 = model_1.metabolites.get_by_id("NAD_p2")
GLYOX_p2 = Metabolite("GLYOX_p2", formula="C2H1O3", name="Glyoxylate", compartment="p", charge=-1)
NADH_p2 = model_1.metabolites.get_by_id("NADH_p2")
reaction4.add_metabolites({
    GLYCOLLATE_p2: -1,
    NAD_p2: -1,
    GLYOX_p2: 1.0,
    NADH_p2: 1.0
})

# adding reactions for tartronate-semialdehyde synthase for night
reaction5 = Reaction('GCL_RXN_p2')
reaction5.name = 'GCL_RXN_p2'
reaction5.subsystem = 'glyoxyllate to TARTRONIC SEMIALDEHYDE'
reaction5.lower_bound = 0
reaction5.upper_bound = 1000
GLYOX_p2 =Metabolite("GLYOX_p2",formula="C2H1O3", name= "Glyoxylate", compartment = "p", charge = "-1")
PROTON_p2 = model_1.metabolites.get_by_id("PROTON_p2")
TARTRONIC_SEMIALDEHYDE_p2 = Metabolite("TARTRONIC_SEMIALDEHYDE_p2",formula="C3H3O4", name= "TARTRONIC_SEMIALDEHYDE", compartment = "p", charge = "-1")
CARBON_DIOXIDE_p2 = model_1.metabolites.get_by_id("CARBON_DIOXIDE_p2")
reaction5.add_metabolites({
    GLYOX_p2: -1,
    PROTON_p2: -1,
    TARTRONIC_SEMIALDEHYDE_p2: 1.0,
    CARBON_DIOXIDE_p2: 1.0,
})

# adding reactions for tartronate-semialdehyde reductase for night
reaction6 = Reaction('RXN_TSR_p2')
reaction6.name = 'RXN_TSR_p2'
reaction6.subsystem = 'TARTRONIC SEMIALDEHYDE to glycerate'
reaction6.lower_bound = -1000
reaction6.upper_bound = 1000
TARTRONIC_SEMIALDEHYDE_p2 = Metabolite("TARTRONIC_SEMIALDEHYDE_p2",formula="C3H3O4", name= "TARTRONIC_SEMIALDEHYDE", compartment = "p", charge = "-1")
NADH_p2 = model_1.metabolites.get_by_id("NADH_p2")
NAD_p2 = model_1.metabolites.get_by_id("NAD_p2")
GLYCERATE_p2 = model_1.metabolites.get_by_id("GLYCERATE_p2")
reaction6.add_metabolites({
    TARTRONIC_SEMIALDEHYDE_p2: -1,
    NADH_p2: -1,
    GLYCERATE_p2: 1.0,
    NAD_p2: 1.0,
})
reactions_to_add.extend([reaction1,reaction2,reaction3, reaction4, reaction5, reaction6])
model_1.add_reactions(reactions_to_add)
# print the reaction for checking the full reaction
# print(reaction1)

#Step:2 optimizing the model by pfba after updates

In [8]:
# run pfba and get summary
solution1 = pfba(model_1)
model_1.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,27.46,1,100.00%
WATER_e1,H2O_tx1,35.59,0,0.00%
NITRATE_e1,Nitrate_tx1,0.3442,0,0.00%
NITRATE_e2,Nitrate_tx2,0.2295,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,2.203,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
SULFATE_e1,SO4_tx1,0.004386,0,0.00%
PROTON_c1,unlProtHYPO_c1,14.22,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e2,CO2_tx2,-2.12,1,100.00%


#Step :3 Appying Pathway 2 to the C3 diel leaf model

Pathway 2 represents AP2 in [South et al.,2019](https://www.science.org/doi/10.1126/science.aat9077) study. The enzymes glycolate oxidase, malate synthase and catalase make this pathway.
*   Making a copy of C3 model to apply AP2
*   Made the copy of C3 model for AP2 by using the syntax `model_2 = C3_model.copy()`

In [9]:
#Making a copy of C3_model for AP2
model_2 = C3_model.copy()

# Disabling the  existing photorespiration pathway before allowing the alternate pathway by applying the following constraints
model_2.reactions.get_by_id("GLYCOLLATE_pc1").bounds = (0,0)
model_2.reactions.get_by_id("GLYCERATE_GLYCOLLATE_pc1").bounds = (0,0)
model_2.reactions.get_by_id("Glycolate_xc1").bounds = (0,0)

### Adding reactions to the model for AP2
*   creating an open list by using `reactions_to_add = []`.
*   Add the reactions in AP2 to both day and night
*   adding the reaction list to the model by `reactions_to_add.extend([R7,R8,R9...])` and `model_name.add_reactions(reactions_to_add)`


In [10]:

from cobra import Model, Reaction, Metabolite
# Create a list to hold reactions
reactions_to_add = []

# adding reactions for glycolate oxidase for day
reaction7 = Reaction('RXN_969_p1')
reaction7.name = 'RXN_969_p1'
reaction7.subsystem = 'glycolate to glyoxyllate'
reaction7.lower_bound = 0
reaction7.upper_bound = 1000
GLYCOLLATE_p1 = model_2.metabolites.get_by_id("GLYCOLLATE_p1")
OXYGEN_MOLECULE_p1 = model_2.metabolites.get_by_id("OXYGEN_MOLECULE_p1")
GLYOX_p1 =Metabolite("GLYOX_p1",formula="C2H1O3", name= "Glyoxylate", compartment = "p", charge = "-1")
HYDROGEN_PEROXIDE_p1 = model_2.metabolites.get_by_id("HYDROGEN_PEROXIDE_p1")
reaction7.add_metabolites({
    GLYCOLLATE_p1: -1,
    OXYGEN_MOLECULE_p1: -1,
    GLYOX_p1: 1.0,
    HYDROGEN_PEROXIDE_p1: 1.0
})

# adding reactions for Malate synthase for day
reaction8 = Reaction('MALSYN_RXN_p1')
reaction8.name = 'MALSYN_RXN_p1'
reaction8.subsystem = 'glyoxyllate to malate'
reaction8.lower_bound = 0
reaction8.upper_bound = 1000
WATER_p1 = model_2.metabolites.get_by_id("WATER_p1")
GLYOX_p1 =Metabolite("GLYOX_p1",formula="C2H1O3", name= "Glyoxylate", compartment = "p", charge = "-1")
ACETYL_COA_p1 = model_2.metabolites.get_by_id("ACETYL_COA_p1")
MAL_p1 = model_2.metabolites.get_by_id("MAL_p1")
CO_A_p1 = model_2.metabolites.get_by_id("CO_A_p1")
PROTON_p1 = model_2.metabolites.get_by_id("PROTON_p1")
reaction8.add_metabolites({
    WATER_p1: -1,
    GLYOX_p1: -1,
    ACETYL_COA_p1: -1,
    MAL_p1: 1.0,
    CO_A_p1: 1.0,
    PROTON_p1: 1.0
})

# adding reactions for glycolate oxidase for night
reaction9 = Reaction('RXN_969_p2')
reaction9.name = 'RXN_969_p2'
reaction9.subsystem = 'glycolate to glyoxyllate'
reaction9.lower_bound = 0
reaction9.upper_bound = 1000
GLYCOLLATE_p2 = model_2.metabolites.get_by_id("GLYCOLLATE_p2")
OXYGEN_MOLECULE_p2 = model_2.metabolites.get_by_id("OXYGEN_MOLECULE_p2")
GLYOX_p2 =Metabolite("GLYOX_p2",formula="C2H1O3", name= "Glyoxylate", compartment = "p", charge = "-1")
HYDROGEN_PEROXIDE_p2 = model_2.metabolites.get_by_id("HYDROGEN_PEROXIDE_p2")
reaction9.add_metabolites({
    GLYCOLLATE_p2: -1,
    OXYGEN_MOLECULE_p2: -1,
    GLYOX_p2: 1.0,
    HYDROGEN_PEROXIDE_p2: 1.0
})

# adding reactions for Malate synthase for night
reaction10 = Reaction('MALSYN_RXN_p2')
reaction10.name = 'MALSYN_RXN_p2'
reaction10.subsystem = 'glyoxyllate to malate'
reaction10.lower_bound = 0
reaction10.upper_bound = 1000
WATER_p2 = model_2.metabolites.get_by_id("WATER_p2")
GLYOX_p2 =Metabolite("GLYOX_p2",formula="C2H1O3", name= "Glyoxylate", compartment = "p", charge = "-1")
ACETYL_COA_p2 = model_2.metabolites.get_by_id("ACETYL_COA_p2")
MAL_p2 = model_2.metabolites.get_by_id("MAL_p2")
CO_A_p2 = model_2.metabolites.get_by_id("CO_A_p2")
PROTON_p2 = model_2.metabolites.get_by_id("PROTON_p2")
reaction10.add_metabolites({
    WATER_p2: -1,
    GLYOX_p2: -1,
    ACETYL_COA_p2: -1,
    MAL_p2: 1.0,
    CO_A_p2: 1.0,
    PROTON_p2: 1.0
})
reactions_to_add.extend([reaction7, reaction8,reaction9,reaction10])
model_2.add_reactions(reactions_to_add)
# print the reaction for checking the full reaction
# print(reaction7)

#Step:4 optimizing the model by pfba after updates

In [11]:
#run pfba and get summary
solution2 = pfba(model_2)
model_2

Name,PlantCoreMetabolism_v2_0_0
Memory address,7987cbb41ed0
Number of metabolites,1771
Number of reactions,1878
Number of genes,0
Number of groups,208
Objective expression,1.0*diel_biomass - 1.0*diel_biomass_reverse_79408
Compartments,"m1, c1, b1, p1, v1, x1, r1, mi1, mc1, e1, l1, i1, c2, v2, e2, x2, b2, p2, m2, mi2, i2, mc2, r2, l2, p"


# Step: 5 Checking the functionality of pathway
Checking the functionality of AP2 by checking the fluxes through the newly added reactions



In [12]:
#Check the reaction fluxes involving pyurvate in plastid during the day
met = model_2.metabolites.PYRUVATE_p1

for rxn in met.reactions:
  print(rxn.id+"\t"+str(rxn.flux))

PEPDEPHOS_RXN_p1	0.0
ACETOLACTSYN_RXN_p1	0.009754750711325361
MALIC_NADP_RXN_p1	0.0
HMBPP_synthesis_p1	0.0
CYSTATHIONINE_BETA_LYASE_RXN_p1	0.0011270124847966886
ANTHRANSYN_RXN_p1	0.0011270124847966882
PYRUVATE_pc1	-0.02791640650553205
ACETOOHBUTSYN_RXN_p1	0.0046908092661575125
PYRUVDEH_RXN_p1	0.0
DIHYDRODIPICSYN_RXN_p1	0.005970120786315266
PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p1	0.0


Checking fluxes through biomass and Phloem_output reactions for understanding the effect of AP2

In [ ]:
# Checking fluxes through biomass reaction
rxn = model_2.reactions.get_by_id("diel_biomass")
print(rxn.flux)

0.6729711992994301


In [ ]:
# Checking fluxes through Phloem output day time
rxn = model_2.reactions.get_by_id("Phloem_output_tx1")
print(rxn.flux)

2.01891359789829


*   From the results, under AP2 or pathway 2, the cycle is not functional

#Step:6 Force cycling of the pathway to make AP2 functional

*  making the same flux for Malic enzyme reaction, pyruvate dehydogenase reaction and Malate synthase reaction

In [ ]:
#making the same flux for the reaction to make the cycle
model_2
same_flux = model_2.problem.Constraint(
    model_2.reactions.MALSYN_RXN_p1.flux_expression - model_2.reactions.MALIC_NADP_RXN_p1.flux_expression,
    lb=0,
    ub=0)
same_flux.id="AP2_cons1"
model_2.add_cons_vars(same_flux)

same_flux = model_2.problem.Constraint(
    model_2.reactions.PYRUVDEH_RXN_p2.flux_expression - model_2.reactions.MALIC_NADP_RXN_p1.flux_expression,
    lb=0,
    ub=0)
same_flux.id="AP2_cons2"
model_2.add_cons_vars(same_flux)

#Step:7 optimizing the model by pfba after updates

In [ ]:
#run pfba and get summary
solution2 = pfba(model_2)
model_2

Name,PlantCoreMetabolism_v2_0_0
Memory address,7c875a7c32d0
Number of metabolites,1771
Number of reactions,1878
Number of genes,0
Number of groups,208
Objective expression,1.0*diel_biomass - 1.0*diel_biomass_reverse_79408
Compartments,"m1, c1, b1, p1, v1, x1, r1, mi1, mc1, e1, l1, i1, c2, v2, e2, x2, b2, p2, m2, mi2, i2, mc2, r2, l2, p"


#Step:8 Checking the fluxes through reactions for comparison
*   Checking the fluxes through the reactions for comparison
*   Checking the biomass rection flux and phloem output fluxes










In [ ]:
met = model_2.metabolites.MAL_p1

for rxn in met.reactions:
  print(rxn.id+"\t"+str(rxn.flux))

2KG_MAL_pc1	-11.253834843571726
OAA_MAL_pc1	-20.17797406491946
MALATE_DEH_RXN_p1	0.0
MALIC_NADP_RXN_p1	11.135949611267144
GLT_MAL_pc1	11.25333393404287
MALATE_DEHYDROGENASE_NADP_RXN_p1	20.178474974448235
MALSYN_RXN_p1	11.135949611267144


In [ ]:
rxn = model_2.reactions.get_by_id("diel_biomass")
print(rxn.flux)

0.3988097628865907


In [ ]:
rxn = model_2.reactions.get_by_id("Phloem_output_tx1")
print(rxn.flux)

1.196429288659772


#Step:9 Constraints to change Vc/Vo ratio

*   These results suggest that the functionality of the cycle reduces the
photosynthetic efficiency of the model
*   For increasing the efficiency we tried to manipulate the Vc/Vo ratio by the following steps, there by tried to match the biomass and phloem_output to the C3_model
*   Vc/Vo ratio defined in the model by Rubisco reaction, we applied a constraint to change the ratio

In [13]:
#creating a backup for making the updates and comparison
backup_model2 = model_2.copy()
Rubisco_balance = Metabolite("rubisco_bal_p1", name = "Weights to balance RuBP carboxygenase oxygenase balance", compartment = "p1")
rxn = backup_model2.reactions.get_by_id("RXN_961_p1")
rxn.add_metabolites({Rubisco_balance:4})
print(rxn.reaction)
# optimizing the model after the updation
solution2 = pfba(backup_model2)

D_RIBULOSE_15_P2_p1 + OXYGEN_MOLECULE_p1 --> CPD_67_p1 + G3P_p1 + 2.0 PROTON_p1 + 7.0 rubisco_bal_p1


#Step:10 Checking the results
*   Checking flux through biomass reaction and phloem output

In [ ]:
rxn = backup_model2.reactions.get_by_id("diel_biomass")
print(rxn.flux)

0.8241606329205255


In [ ]:
rxn = backup_model2.reactions.get_by_id("Phloem_output_tx1")
print(rxn.flux)

2.4724818987615764


- From the results it suggest that we can achieve the higher photosynthetic efficiency by allowing this pathway with flexible Vc/Vo ratio
- Since the pathway releasing more CO2 and changing the Vc/Vo ratio.

#Step :11 Appying Pathway 3 to the C3 diel leaf model

Pathway 3 represents AP3 in [South et al.,2019](https://www.science.org/doi/10.1126/science.aat9077) study. The enzymes glycolate dehydrogenase, malate synthase  make this pathway.
*   Making a copy of C3 model to apply AP3
*   Made the copy of C3 model for AP3 by using the syntax `model_3 = C3_model.copy()`

In [ ]:
# Making a copy of C3_model for AP3
model_3 = C3_model.copy()

# Disabling the  existing photorespiration pathway before allowing the alternate pathway by applying the following constraints
model_3.reactions.get_by_id("GLYCOLLATE_pc1").bounds = (0,0)
model_3.reactions.get_by_id("GLYCERATE_GLYCOLLATE_pc1").bounds = (0,0)
model_3.reactions.get_by_id("Glycolate_xc1").bounds = (0,0)

### Adding reactions to the model for AP2
*   creating an open list by using `reactions_to_add = []`.
*   Add the reactions in AP3 to both day and night
*   adding the reaction list to the model by `reactions_to_add.extend([R11,R12,R13...])` and `model_name.add_reactions(reactions_to_add)`

In [ ]:

from cobra import Model, Reaction, Metabolite
# Create a list to hold reactions
reactions_to_add = []

# adding reactions for glycolate dehydrogenase for day
reaction11 = Reaction('RXN_GDH_p1')
reaction11.name = 'RXN_GDH_p1'
reaction11.subsystem = 'glycolate to glyoxyllate by NAD'
reaction11.lower_bound = 0
reaction11.upper_bound = 1000
GLYCOLLATE_p1 = model_3.metabolites.get_by_id("GLYCOLLATE_p1")
NAD_p1 = model_3.metabolites.get_by_id("NAD_p1")
GLYOX_p1 =Metabolite("GLYOX_p1",formula="C2H1O3", name= "Glyoxylate", compartment = "p", charge = "-1")
NADH_p1 = model_3.metabolites.get_by_id("NADH_p1")
reaction11.add_metabolites({
    GLYCOLLATE_p1: -1,
    NAD_p1: -1,
    GLYOX_p1: 1.0,
    NADH_p1: 1.0
})

# adding reactions for Malate synthase for day
reaction12 = Reaction('MALSYN_RXN_p1')
reaction12.name = 'MALSYN_RXN_p1'
reaction12.subsystem = 'glyoxyllate to malate'
reaction12.lower_bound = 0
reaction12.upper_bound = 1000
WATER_p1 = model_3.metabolites.get_by_id("WATER_p1")
GLYOX_p1 =Metabolite("GLYOX_p1",formula="C2H1O3", name= "Glyoxylate", compartment = "p", charge = "-1")
ACETYL_COA_p1 = model_3.metabolites.get_by_id("ACETYL_COA_p1")
MAL_p1 = model_3.metabolites.get_by_id("MAL_p1")
CO_A_p1 = model_3.metabolites.get_by_id("CO_A_p1")
PROTON_p1 = model_3.metabolites.get_by_id("PROTON_p1")
reaction12.add_metabolites({
    WATER_p1: -1,
    GLYOX_p1: -1,
    ACETYL_COA_p1: -1,
    MAL_p1: 1.0,
    CO_A_p1: 1.0,
    PROTON_p1: 1.0
})

# adding reactions for glycolate dehydrogenase for night
reaction13 = Reaction('RXN_GDH_p2')
reaction13.name = 'RXN_GDH_p2'
reaction13.subsystem = 'glycolate to glyoxyllate by NAD'
reaction13.lower_bound = 0
reaction13.upper_bound = 1000
GLYCOLLATE_p2 = model_3.metabolites.get_by_id("GLYCOLLATE_p2")
NAD_p2 = model_3.metabolites.get_by_id("NAD_p2")
GLYOX_p2 =Metabolite("GLYOX_p2",formula="C2H1O3", name= "Glyoxylate", compartment = "p", charge = "-1")
NADH_p2 = model_3.metabolites.get_by_id("NADH_p2")
reaction13.add_metabolites({
    GLYCOLLATE_p2: -1,
    NAD_p2: -1,
    GLYOX_p2: 1.0,
    NADH_p2: 1.0
})

# adding reactions for Malate synthase for night
reaction14 = Reaction('MALSYN_RXN_p2')
reaction14.name = 'MALSYN_RXN_p2'
reaction14.subsystem = 'glyoxyllate to malate'
reaction14.lower_bound = 0
reaction14.upper_bound = 1000
WATER_p2 = model_3.metabolites.get_by_id("WATER_p2")
GLYOX_p2 =Metabolite("GLYOX_p2",formula="C2H1O3", name= "Glyoxylate", compartment = "p", charge = "-1")
ACETYL_COA_p2 = model_3.metabolites.get_by_id("ACETYL_COA_p2")
MAL_p2 = model_3.metabolites.get_by_id("MAL_p2")
CO_A_p2 = model_3.metabolites.get_by_id("CO_A_p2")
PROTON_p2 = model_3.metabolites.get_by_id("PROTON_p2")
reaction14.add_metabolites({
    WATER_p2: -1,
    GLYOX_p2: -1,
    ACETYL_COA_p2: -1,
    MAL_p2: 1.0,
    CO_A_p2: 1.0,
    PROTON_p2: 1.0
})


reactions_to_add.extend([reaction11, reaction12,reaction13,reaction14])
model_3.add_reactions(reactions_to_add)
# print the reaction for checking the full reaction
# print(reaction11)
# print(reaction12)

RXN_GDH_p1: GLYCOLLATE_p1 + NAD_p1 --> GLYOX_p1 + NADH_p1
MALSYN_RXN_p1: ACETYL_COA_p1 + GLYOX_p1 + WATER_p1 --> CO_A_p1 + MAL_p1 + PROTON_p1


#Step:12 optimizing the model by pfba after updates

In [ ]:
#run pfba and get summary
solution3 = pfba(model_3)
model_3

Name,PlantCoreMetabolism_v2_0_0
Memory address,7c8758376ad0
Number of metabolites,1771
Number of reactions,1878
Number of genes,0
Number of groups,208
Objective expression,1.0*diel_biomass - 1.0*diel_biomass_reverse_79408
Compartments,"m1, c1, b1, p1, v1, x1, r1, mi1, mc1, e1, l1, i1, c2, v2, e2, x2, b2, p2, m2, mi2, i2, mc2, r2, l2, p"


#Step: 13 Checking the functionality of pathway
Checking the functionality of AP3 by checking the fluxes through the newly added reactions

In [ ]:
##Check the reaction fluxes involving pyurvate in plastid during the day
met = model_3.metabolites.PYRUVATE_p1

for rxn in met.reactions:
  print(rxn.id+"\t"+str(rxn.flux))

MALIC_NADP_RXN_p2	0.0
CYSTATHIONINE_BETA_LYASE_RXN_p2	0.0
ANTHRANSYN_RXN_p2	0.0
ACETOOHBUTSYN_RXN_p2	0.0
PYRUVATE_pc2	-0.006033769558419601
PYRUVDEH_RXN_p2	0.001610199941953849
DIHYDRODIPICSYN_RXN_p2	0.0
PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p2	0.0
PEPDEPHOS_RXN_p2	0.003131983999419367
ACETOLACTSYN_RXN_p2	0.003777776807942229
HMBPP_synthesis_p2	0.0


Checking fluxes through biomass and Phloem_output reactions for understanding the effect of AP2

In [ ]:
# Checking fluxes through biomass reaction
rxn = model_3.reactions.get_by_id("diel_biomass")
print(rxn.flux)

0.7818759487645678


In [ ]:
# Checking fluxes through biomass Phloem_output day
rxn = model_3.reactions.get_by_id("Phloem_output_tx1")
print(rxn.flux)

2.345627846293703


*   From the results, under AP3 or pathway 3, the cycle is not functional

#Step:14 Force cycling of the pathway to make AP3 functional

*   making the same flux for Malic enzyme reaction, pyruvate dehydogenase reaction and Malate synthase reaction

In [ ]:
# making the reactions have the same flux to make the cycle
model_3
same_flux = model_3.problem.Constraint(
    model_3.reactions.MALSYN_RXN_p1.flux_expression - model_3.reactions.MALIC_NADP_RXN_p1.flux_expression,
    lb=0,
    ub=0)
same_flux.id="AP2_cons1"
model_3.add_cons_vars(same_flux)

same_flux = model_3.problem.Constraint(
    model_3.reactions.PYRUVDEH_RXN_p2.flux_expression - model_3.reactions.MALIC_NADP_RXN_p1.flux_expression,
    lb=0,
    ub=0)
same_flux.id="AP2_cons2"
model_3.add_cons_vars(same_flux)

#Step:15 optimizing the model by pfba after updates

In [ ]:
#run pfba and get summary
solution3 = pfba(model_3)
model_3

Name,PlantCoreMetabolism_v2_0_0
Memory address,7c8758376ad0
Number of metabolites,1771
Number of reactions,1878
Number of genes,0
Number of groups,208
Objective expression,1.0*diel_biomass - 1.0*diel_biomass_reverse_79408
Compartments,"m1, c1, b1, p1, v1, x1, r1, mi1, mc1, e1, l1, i1, c2, v2, e2, x2, b2, p2, m2, mi2, i2, mc2, r2, l2, p"


#Step:16 Checking the fluxes through reactions for comparison
*   Checking the fluxes through the reactions for comparison
*   Checking the biomass rection flux and phloem output fluxes


In [ ]:
rxn = model_3.reactions.get_by_id("diel_biomass")
print(rxn.flux)

0.4619398358474981


In [ ]:
rxn = model_3.reactions.get_by_id("Phloem_output_tx1")
print(rxn.flux)

1.385819507542494


#Step:17 Constraints to change Vc/Vo ratio

*   These results suggest that the functionality of the cycle reduces the
photosynthetic efficiency of the model
*   For increasing the efficiency we tried to manipulate the Vc/Vo ratio by the following steps, there by tried to match the biomass and phloem_output to the C3_model
*   Vc/Vo ratio defined in the model by Rubisco reaction, we applied a constraint to change the ratio

In [ ]:
#creating a backup for making the updates and comparison
backup_model3 = model_3.copy()
Rubisco_balance = Metabolite("rubisco_bal_p1", name = "Weights to balance RuBP carboxygenase oxygenase balance", compartment = "p1")
rxn = backup_model3.reactions.get_by_id("RXN_961_p1")
rxn.add_metabolites({Rubisco_balance:3.5})
print(rxn.reaction)
solution3 = pfba(backup_model3)

D_RIBULOSE_15_P2_p1 + OXYGEN_MOLECULE_p1 --> CPD_67_p1 + G3P_p1 + 2.0 PROTON_p1 + 6.5 rubisco_bal_p1


#Step:18 Checking the results
*   Checking flux through biomass reaction and phloem *output*

In [ ]:
rxn = backup_model3.reactions.get_by_id("diel_biomass")
print(rxn.flux)

0.8606067400347465


In [ ]:
rxn = backup_model3.reactions.get_by_id("Phloem_output_tx1")
print(rxn.flux)

2.5818202201042393


- From the results it suggest that we can achieve the higher photosynthetic efficiency by allowing this pathway with flexible Vc/Vo ratio
- Since the pathway releasing more CO2 and changing the Vc/Vo ratio.

# ***Part:3 Comparing results ***
Comparing the different pathway results by checking the Biomass and phloem_output reaction fluxes

###NOTE:
*   C3_model represent the PlantCoreMetabolism model
*   model_1 represent the model with pathway1
*   model_2 represent the model with pathway2
*   model_3 represent the model with pathway3




In [ ]:
#Checking the flux for biomass under different conditions
rxn = C3_model.reactions.get_by_id("diel_biomass")
print(rxn.id+"\t"+str(rxn.flux))
rxn = model_1.reactions.get_by_id("diel_biomass")
print(rxn.id+"\t"+str(rxn.flux))
rxn = model_2.reactions.get_by_id("diel_biomass")
print(rxn.id+"\t"+str(rxn.flux))
rxn = backup_model2.reactions.get_by_id("diel_biomass")
print(rxn.id+"\t"+str(rxn.flux))
rxn = model_3.reactions.get_by_id("diel_biomass")
print(rxn.id+"\t"+str(rxn.flux))
rxn = backup_model3.reactions.get_by_id("diel_biomass")
print(rxn.id+"\t"+str(rxn.flux))

diel_biomass	0.8258313952247737
diel_biomass	1.3096037769153317
diel_biomass	0.3988097628865907
diel_biomass	0.8241606329205255
diel_biomass	0.4619398358474981
diel_biomass	0.8606067400347465


In [ ]:
#Checking the flux for phloem output under different conditions
rxn = C3_model.reactions.get_by_id("Phloem_output_tx1")
print(rxn.id+"\t"+str(rxn.flux))
rxn = model_1.reactions.get_by_id("Phloem_output_tx1")
print(rxn.id+"\t"+str(rxn.flux))
rxn = model_2.reactions.get_by_id("Phloem_output_tx1")
print(rxn.id+"\t"+str(rxn.flux))
rxn = backup_model2.reactions.get_by_id("Phloem_output_tx1")
print(rxn.id+"\t"+str(rxn.flux))
rxn = model_3.reactions.get_by_id("Phloem_output_tx1")
print(rxn.id+"\t"+str(rxn.flux))
rxn = backup_model3.reactions.get_by_id("Phloem_output_tx1")
print(rxn.id+"\t"+str(rxn.flux))

Phloem_output_tx1	2.477494185674321
Phloem_output_tx1	3.928811330745994
Phloem_output_tx1	1.196429288659772
Phloem_output_tx1	2.4724818987615764
Phloem_output_tx1	1.385819507542494
Phloem_output_tx1	2.5818202201042393


###Inference:
*   From the results it suggest that we can achieve the higher photosynthetic efficiency by allowing this pathway with flexible Vc/Vo ratio
*   AP3 showed maximum photosynthetic efficiency which is similar to the observation from [South et al.,2019](https://www.science.org/doi/10.1126/science.aat9077)

